In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid
import tensorflow as tf

# Load and preprocess Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Grid search parameters
param_grid = {
    'batch_size': [64, 128],
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [0.01, 0.001]
}

best_val_acc = 0
best_params = {}

for params in ParameterGrid(param_grid):
    tf.keras.backend.clear_session()

    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(10, activation='softmax')
    ])

    if params['optimizer'] == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    else:
        opt = tf.keras.optimizers.SGD(learning_rate=params['learning_rate'])

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0),
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
    ]

    history = model.fit(
        x_train, y_train_cat,
        validation_split=0.2,
        batch_size=params['batch_size'],
        epochs=15,
        verbose=0,
        callbacks=callbacks
    )

    val_acc = max(history.history['val_accuracy'])
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_params = params
        best_model = model
        best_history = history

# Evaluate best model
test_loss, test_acc = best_model.evaluate(x_test, y_test_cat, verbose=0)
y_pred = np.argmax(best_model.predict(x_test), axis=1)

# Confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix on Fashion MNIST")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Subplot for training/validation loss and accuracy
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs[0].plot(best_history.history['loss'], label='Train Loss')
axs[0].plot(best_history.history['val_loss'], label='Val Loss')
axs[0].set_title("Loss over Epochs")
axs[0].legend()

axs[1].plot(best_history.history['accuracy'], label='Train Accuracy')
axs[1].plot(best_history.history['val_accuracy'], label='Val Accuracy')
axs[1].set_title("Accuracy over Epochs")
axs[1].legend()

plt.tight_layout()
plt.show()

# Misclassified images
misclassified = np.where(y_pred != y_test)[0][:10]
plt.figure(figsize=(15, 5))
for i, idx in enumerate(misclassified):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_test[idx], cmap='gray')
    plt.title(f"True: {y_test[idx]}, Pred: {y_pred[idx]}")
    plt.axis('off')
plt.suptitle("Sample Misclassified Images")
plt.show()

# Learning rate visualization (simulation based on ReduceLROnPlateau logic)
initial_lr = best_params['learning_rate']
val_loss = best_history.history['val_loss']
lr_schedule = [initial_lr]

for i in range(1, len(val_loss)):
    if val_loss[i] > val_loss[i-1]:
        lr_schedule.append(lr_schedule[-1] * 0.5)
    else:
        lr_schedule.append(lr_schedule[-1])

plt.figure(figsize=(8, 4))
plt.plot(lr_schedule, marker='o')
plt.title("Learning Rate Schedule Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")
plt.grid(True)
plt.show()

# Final report
print("Best Hyperparameters:")
print(best_params)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")




KeyboardInterrupt: 